In [ ]:
# Fix for pyshark event loop issues in Jupyter
import asyncio
import nest_asyncio
nest_asyncio.apply()

import pyshark
from collections import Counter
import polars as pl
import os

# Define the path to the capture file.
# This adjusts for running the notebook from the analysis/notebooks directory.
pcap_file = '../../captures/wireshark/orig_with_pd.13.pcapng'

print(f"Attempting to open: {pcap_file}")
print(f"Current working directory: {os.getcwd()}")

if not os.path.exists(pcap_file):
    print(f"Error: Capture file not found at '{pcap_file}'.")
    print("Please ensure the notebook is run from the root of the 'km003c-protocol-research' directory, or adjust the 'pcap_file' path.")
else:
    print("File found. Processing...")
    
    try:
        # Open the capture file with explicit event loop handling
        cap = pyshark.FileCapture(pcap_file, use_ek=True, include_raw=True)

        packet_count = 0
        protocol_counter = Counter()
        
        # Iterate through all packets with progress reporting
        for packet in cap:
            packet_count += 1
            if packet_count % 100 == 0:  # Progress update every 100 packets
                print(f"Processed {packet_count} packets...")
            
            if hasattr(packet, 'layers'):
                for layer in packet.layers:
                    protocol_counter[layer.layer_name] += 1
        
        cap.close()

        print(f"\n--- Analysis Results ---")
        print(f"Total packets processed: {packet_count}")

        if packet_count > 0:
            print("\nProtocol summary:")
            # Create a DataFrame for nice printing
            protocol_df = pl.DataFrame({
                "Protocol": list(protocol_counter.keys()),
                "Packet Count": list(protocol_counter.values())
            }).sort("Packet Count", descending=True)
            print(protocol_df)
        else:
            print("No packets found in the capture file.")
            
    except Exception as e:
        print(f"Error processing capture file: {e}")
        import traceback
        traceback.print_exc()

Attempting to open: ../../captures/wireshark/orig_with_pd.13.pcapng
Current working directory: /home/okhsunrog/code/km003c-protocol-research/analysis/notebooks
File found. Processing...
Processed 100 packets...
Processed 200 packets...
Processed 300 packets...
Processed 400 packets...
Processed 500 packets...
Processed 600 packets...
Processed 700 packets...
Processed 800 packets...
Processed 900 packets...
Processed 1000 packets...
Processed 1100 packets...
Processed 1200 packets...
Processed 1300 packets...
Processed 1400 packets...
Processed 1500 packets...
Processed 1600 packets...
Processed 1700 packets...
Processed 1800 packets...
Processed 1900 packets...
Processed 2000 packets...

--- Analysis Results ---
Total packets processed: 2056

Protocol summary:
shape: (17, 2)
┌─────────────────────────────────┬──────────────┐
│ Protocol                        ┆ Packet Count │
│ ---                             ┆ ---          │
│ str                             ┆ i64          │
╞════════